In [5]:
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantFormat, QuantType
import tensorflow as tf
import onnxruntime
import numpy as np


c:\Users\jorge.ruiz\Documents\venvs\movenet\lib\site-packages\onnx\mapping.py:27: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  int(TensorProto.STRING): np.dtype(np.object)


In [6]:
import tensorflow as tf
def h5_to_tf(path="./mnist_medium_complexity_float32_sigmoid.h5"):
    model = tf.keras.models.load_model(path)
    model.save("pb_model")

#h5_to_tf()

In [23]:
IN = "./fp32/mnist_sigmoid_fp32.onnx"
OUT = "./uint8/mnist_sigmoid_uint8.onnx"

In [24]:
class MnistDataReader(CalibrationDataReader):
    def __init__(self,):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        self.x=  x_train.reshape((x_train.shape[0], x_train.shape[1]*x_train.shape[2])).astype("float32") / 255

        self.__next_yield = 0

    def get_next(self):
        session = onnxruntime.InferenceSession(IN, None)
        input_name = session.get_inputs()[0].name

        if self.__next_yield >= len(self.x):
            return None

        x = self.x[self.__next_yield]
        self.__next_yield += 1
        return {input_name: np.expand_dims(x, axis=0)}



In [25]:
dr = MnistDataReader()
print(list(dr.get_next().values())[0].shape)
quantize_static(IN, OUT, dr)

(1, 784)
